In [37]:
from cdsfm.encodon import AutoEnCodon
from cdsfm.encodon.modeling_encodon import EnCodonForSequenceTask
from cdsfm.encodon.configuration_encodon import EnCodonForSequenceTaskConfig
import pandas as pd
from datasets import Dataset

In [15]:
automodel = AutoEnCodon.from_pretrained("goodarzilab/encodon-620M-euk")

You are using a model of type encodon to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [17]:
tokenizer = automodel.tokenizer

In [10]:
model = automodel.model

In [12]:
model.save_pretrained('EnCodon-620M-euk')

In [22]:
data = pd.read_csv("../CDS-LM/data/finetuning/saluki/human_sss_reprocessed.csv", index_col=0)


print("\nTokenizing the datasets...")

dataset = Dataset.from_pandas(pd.DataFrame({"seq": data['CDS'].values, 'y': data['y'].values}))

dataset = dataset.map(lambda x: tokenizer(x['seq'],
        #[f"{tokenizer.cls_token}{y}{tokenizer.sep_token}" for y in x["seq"]],
        add_special_tokens=True,
        return_attention_mask=True,
        return_special_tokens_mask=True,
        truncation=True,
        padding=True,
    ),
    batched=True, num_proc=32, remove_columns=["seq"]
)


Tokenizing the datasets...


Map (num_proc=32):   0%|          | 0/12914 [00:00<?, ? examples/s]

In [24]:
dataset

Dataset({
    features: ['y', 'input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 12914
})

In [30]:
input_lens = map(lambda x: len(x), dataset['token_type_ids'])

In [35]:
model = EncodonForSequenceTask

EnCodon(
  (bert): EnCodonModule(
    (embeddings): EnCodonEmbeddings(
      (word_embeddings): Embedding(69, 2048, padding_idx=3)
      (position_embeddings): Embedding(2048, 2048)
      (token_type_embeddings): Embedding(2, 2048)
      (LayerNorm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): EnCodonStack(
      (layer): ModuleList(
        (0-11): 12 x EnCodonLayer(
          (attention): EnCodonAttention(
            (pre_layer_norm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
            (post_attn_dense): Linear(in_features=2048, out_features=2048, bias=True)
            (post_layer_norm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (self_attention): MultiHeadedSelfAttention(
              (query): Linear(in_features=2048, out_features=2048, bias=True)
              (key): Linear(in_features=2048, out_features=2048, 

In [44]:
config = EnCodonForSequenceTaskConfig.from_pretrained("EnCodon-620M-euk")

In [46]:
config.problem_type = 'regression'
config.cls_type = 'cls'
config.num_labels = 1
config.num_tasks = 1
config.classifier_dropout = 0.3
config.attention_probs_dropout_prob=0.0
config.hidden_dropout_prob=0.0
config.loss_fn = 'huber'

In [47]:
config

EnCodonForSequenceTaskConfig {
  "architectures": [
    "EnCodon"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "self",
  "auto_map": {
    "AutoConfig": "goodarzilab/encodon-620M-euk--configuration_encodon.EnCodonConfig",
    "AutoModelForMaskedLM": "goodarzilab/encodon-620M-euk--modeling_encodon.EnCodon",
    "AutoModelForSequenceClassification": "goodarzilab/encodon-620M-euk--cdsfm.encodon.modeling_encodon.EnCodonForSequenceTask",
    "AutoTokenizer": "goodarzilab/encodon-620M-euk--cdsfm.encodon.tokenization_encodon.EnCodonTokenizer"
  },
  "classifier_dropout": 0.3,
  "cls_dropout_prob": 0.1,
  "cls_hidden_act": "relu",
  "cls_hidden_size": 128,
  "cls_num_attention_heads": 8,
  "cls_num_hidden_layers": 1,
  "cls_rotary_theta": 10000.0,
  "cls_type": "cls",
  "cls_use_rotary_emb": false,
  "dilation": 1,
  "dilation_rates": null,
  "gamma_init": 1.782709687623856,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 2048,
  "id2label": {
    "0":

In [42]:
model2 = EnCodonForSequenceTask(config)

In [43]:
model2

EnCodonForSequenceTask(
  (bert): EnCodonModule(
    (embeddings): EnCodonEmbeddings(
      (word_embeddings): Embedding(69, 2048, padding_idx=3)
      (position_embeddings): Embedding(2048, 2048)
      (token_type_embeddings): Embedding(2, 2048)
      (LayerNorm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): EnCodonStack(
      (layer): ModuleList(
        (0-11): 12 x EnCodonLayer(
          (attention): EnCodonAttention(
            (pre_layer_norm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
            (post_attn_dense): Linear(in_features=2048, out_features=2048, bias=True)
            (post_layer_norm): LayerNorm((2048,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (self_attention): MultiHeadedSelfAttention(
              (query): Linear(in_features=2048, out_features=2048, bias=True)
              (key): Linear(in_features=2048, out_